# Challenge 8: 自駕車接收命令與指揮 (計算理論 / 系統軟體)

2020.11.27

## Objective

- Learn language grammar to create own language which handled by specific application
- Understand regular language and context free language
- Learn useful command-line skill to leverage various system software
- Understand how application leverage system software

> TODO: language capability circle graph

## Requirement

實作出小鴨車語 `DuckieDuckie2` 直譯器，可利用環境變數調整速度與角速度，且可從網路取得程式碼

- 小鴨車語 `DuckieDuckie2`
    - 從 regular language 的 `DuckieDuckie1` 進展到 context free language 的 `DuckieDuckie2`
    - 新增迴圈
- 利用環境變數去調整 `前進` `後退` `右轉` `左轉` 時的速度與角速度
    - 方便在不修改程式碼的情況下，改變與設定程式的行為
- 從網路取得程式碼
    - 安裝與使用 `nc` 工具
    - 先使用 `nc` 透過 `TCP` 取得程式碼並將輸出重定向到檔案
    - 在透過輸入重定向將程式碼檔案餵給 `DuckieDuckie2` 直譯器


### 小鴨車語 `DuckieDuckie2`

#### 語法 Grammar

小鴨車語 `DuckieDuckie2` 的上下文語法定義為

$$
\begin{align}
G = \{ \sum, N, S, P \} \\
G \space \text{is grammar} \\
\sum \space \text{is alphabet and terminal symbol} \\
N \space \text{is nonterminal symbol} \\
S \space \text{is start symbol} \\
P \space \text{is production rules}
\end{align}
$$

- `alphabet` 字母集為 `a-z`, `0-9`, `;`, 換行 `\n` 與 空白 ` ` ，亦指 `terminal symbol` 終端符號
- `nonterminal symbol` 非終端符號有 `CMD`, `DIGIT`, `SCALAR`, `UNIT`, `TRIM`, `LOOP`, `DUCKIEDUCKIE1`, `DUCKIEDUCKIE2`
- `start symbol` 開始符號為 `DUCKIEDUCKIE2`
- `production rules` 產生規則較為複雜，其中使用到幾個符號
  - 使用三個正則運算：
    - 用 $\cup$ 表示 Union 聯集
    - 用 $\cdot$ 表示 Concatenate 連接
    - 用 $*$ 表示 Kleene Star 星號  
  - 使用單引號 $\text{'}$ 表達字母集中的終端符號  
  - 沒有單引號也非正則運算表達非終端符號

$$
\begin{align}
\text{CMD} &\to \text{'forward'} \cup \text{'backward'} \cup \text{'left'} \cup \text{'right'} & \\
\text{DIGIT} &\to \text{'0'} \cup \text{'1'} \
\cup \text{'2'} \cup \text{'3'} \cup \text{'4'} \cup \text{'5'} \
\cup \text{'6'} \cup \text{'7'} \cup \text{'8'} \cup \text{'9'} & \\
\text{SCALAR} &\to \text{DIGIT} \cdot \text{DIGIT}^* & \\
\text{UNIT} &\to \text{'ms'} \cup \text{'s'} & \\
\text{TRIM} &\to \text{' '} \cup \text{'\\n'} & \\
\text{LOOP} &\to \text{'loop'} & \\
\\
\text{DUCKIEDUCKIE1} &\to \text{CMD} \cdot \text{' '} \cdot \text{' '}^* \cdot \text{SCALAR} \cdot \text{UNIT} \cdot \text{';'} \cdot \text{TRIM}^* & \\
\text{DUCKIEDUCKIE2} &\to \text{DUCKIEDUCKIE1}^* \cup \text{LOOP} \cdot \text{' '} \cdot \text{SCALAR} \cdot \text{' '} \cdot \text{'\{'} \cdot \text{TRIM}^* \cdot \text{DUCKIEDUCKIE2} \cdot \text{DUCKIEDUCKIE2} \cdot \text{'\}'} \cdot \text{TRIM}^*
\end{align}
$$


#### 語義 Semantics

`DuckieDuckie1` 的非終端符號會被分成四個字串，分別是 `CMD`、`SCALAR`、`UNIT` 與 `TRIM`

- `CMD` 對應到小鴨車應該要有的行為
  - 目前 `DuckieDuckie1` 有四種行為
    - `forward` 前進
    - `backward` 後退
    - `left` 左轉
    - `right` 右轉
- `SCALAR` 對應到 `CMD` 行為持續時間的多寡，與 `UNIT` 配合表示不同量級的參數
- `UNIT` 對應到 `SCALAR` 的單位
- `TRIM` 在語義上沒有意義，應該忽略

其對應語義為：控制小鴨車進行 `CMD` 行為，持續時間為 `SCALAR`，時間單位為 `UNIT`

舉例而言，當 `CMD='forward', SCALAR='1', UNIT='s'`  
對應語義即為，小鴨車前進一秒

`DuckieDuckie2` 有兩種情況

1. 多個 `DuckieDuckie1`
  - 按照順序，等前一個 `DuckieDuckie1` 的語義結束後，處理下一個

舉例而言，當遇到 `CMD='forward', SCALAR='1', UNIT='s'` 接著 `CMD='backward', SCALAR='1', UNIT='s'`
對應語義即為，小鴨車前進一秒，接著後退一秒

2. 遇到 `LOOP` 迴圈
  - 後面接著的 `SCALAR` 代表該迴圈要重複幾次
  - 重複的部分為被 `{` 與 `}` 包圍起來的部分

舉例而言，當遇到 `SCALAR=2`，且 `{` `}` 包圍的部分是  `CMD='forward', SCALAR='1', UNIT='s'` 接著 `CMD='right', SCALAR='1', UNIT='s'`
對應語義即為，小鴨車前進一秒，接著右轉一秒，再前進一秒，最後右轉一秒

## Test Example

小鴨車語 `DuckieDuckie2` 的使用範例與行為

有一個檔案 `main.duckie`
```
forward 1500ms;
right 1s;
```

透過命令列 `python duckieduckie2.py < main.duckie` 使用輸入重定向
> 但在小鴨車環境上的話，已經是 `bash` 所以可以直接使用輸入重定向

預期會看到小鴨車的行為是：

```
前進 1500 毫秒，右轉 1 秒，最後停止
```


### Example 1: 左轉 3 秒，重複 2 次

檔案 `example1.duckie`
```
loop 2 {
    left 3s;
}
```
連入小鴨車，並使用  
`python duckieduckie2.py < example1.duckie`  
會看到小鴨車行為是：
```
左轉 3 秒，再左轉 3 秒，然後停止
```

### Example 2: 前進 1 秒，右轉 1 秒，重複 4 次

檔案 `example2.duckie`
```
loop 4 {
    forward 1s;
    right 1s;
}
```
連入小鴨車，並使用  
`python duckieduckie2.py < example2.duckie`  
會看到小鴨車行為是：
```
前進 1 秒，右轉 1 秒，前進 1 秒，右轉 1 秒，前進 1 秒，右轉 1 秒，前進 1 秒，右轉 1 秒，最後停止
```


### Example 3: 巢狀迴圈

檔案 `example3.duckie`
```
loop 2 {
    loop 4 {
        forward 1s;
        left 1s;
    }
    right 2s;
}

```
連入小鴨車，並使用  
`python duckieduckie2.py < example3.duckie`  
會看到小鴨車行為是：
```
前進 1 秒，左轉 1 秒，重複 4 次
且上述行為重複 2 次
```

### Example 4: 語法錯誤 

檔案 `example3.duckie`
```
loop 4 {
    forward 1s;
    left 1s;
}
right 2s;
```
連入小鴨車，並使用  
`python duckieduckie2.py < example4.duckie`  
會看到小鴨車行為是：
```
停止
```

因為**語法是錯的，應該提示使用者語法有誤**

## Notice

- 因需要使用到 `duckiebot_cs_zoo/libs` 目錄下的函式庫，請將程式碼放置於 `duckiebot_cs_zoo` 目錄下
- 有任何問題，歡迎發問